In [146]:
import pandas as pd 
import os
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

#### Loading data

In [35]:
csv_file_path = os.path.join("..", "data", "IMDB Dataset.csv")
df = pd.read_csv(csv_file_path)


In [94]:
X = df['review']  
y = df['sentiment'] 

# Division en parties d'entraînement (80%) et de validation (20%)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

print("Taille de l'ensemble d'entraînement :", len(X_train))
print("Taille de l'ensemble de validation :", len(X_valid))

Taille de l'ensemble d'entraînement : 35000
Taille de l'ensemble de validation : 15000


In [95]:
X_train.reset_index(drop=True, inplace=True)


In [100]:
X_train_df = pd.DataFrame({'review': X_train}, columns=['review'])

X_valid_df = pd.DataFrame({'review': X_valid}, columns=['review'])

y_train_df = pd.DataFrame({'sentiment': y_train}, columns=['sentiment'])

y_valid_df = pd.DataFrame({'sentiment': y_valid}, columns=['sentiment'])

#### Preprocessing : label distribution , tokenization and lemmatisation

In [47]:
positive_label_count  = df[df["sentiment"] == "positive"].shape[0]
negative_label_count = df[df["sentiment"] == "negative"].shape[0]

In [49]:
positive_label_proportion = (positive_label_count / (positive_label_count+ negative_label_count))*100

In [50]:
negative_label_proportion= (negative_label_count / (positive_label_count+ negative_label_count))*100

In [52]:
print('la proportion de sentiments positives est de ', positive_label_proportion)
print('la proportion de sentiments négatifs est de ', negative_label_proportion)

la proportion de sentiments positives est de  50.0
la proportion de sentiments négatifs est de  50.0


In [53]:
print("on constate on a autant de labels positifs que de labels négatifs")

on constate on a autant de labels positifs que de labels négatifs


In [72]:
import spacy

try:
    #rem: enable does not seem to work, don't know why
    nlp = spacy.load("en_core_web_sm") #, enable=["tok2vec", "morphologizer", "lemmatizer"])
except OSError:
    !python -m spacy download fr_core_news_md
    nlp = spacy.load("en_core_web_sm") #, enable=["tok2vec", "morphologizer", "lemmatizer"])

# we won't need ner, parser, attribute_ruler
# NB: tok2vec and morphologizer seem to be necessary for lemmatization
nlp.remove_pipe("ner")
nlp.remove_pipe("parser")
nlp.remove_pipe("attribute_ruler")
print('Current pipeline:\n  '+'\n  '.join(nlp.pipe_names))


Current pipeline:
  tok2vec
  tagger
  lemmatizer


In [112]:
def tokenize_text(text) :
    doc = nlp(text)
    tokens =[token.text for token in doc ]
    return tokens

def lemmatize_text(text) :
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc ]
    return lemmas 
    

In [137]:
X_valid_df['tokens'] = X_train_df['review'].apply(tokenize_text)

/home/moustapha/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [113]:
%%time
X_train_df['lemmas'] = X_train_df['review'].apply(lemmatize_text)

/home/moustapha/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


CPU times: user 7min 55s, sys: 5.97 s, total: 8min 1s
Wall time: 8min 2s


In [114]:
X_train_df

,review,tokens,lemmas
0,"As much as I love trains, I couldn't stomach t...","[As, much, as, I, love, trains, ,, I, could, n...","[as, much, as, i, love, trains, ,, i, could, n..."
1,"This was a very good PPV, but like Wrestlemani...","[This, was, a, very, good, PPV, ,, but, like, ...","[this, was, a, very, good, ppv, ,, but, like, ..."
2,Not finding the right words is everybody's pro...,"[Not, finding, the, right, words, is, everybod...","[not, finding, the, right, words, is, everybod..."
3,I'm really suprised this movie didn't get a hi...,"[I, 'm, really, suprised, this, movie, did, n'...","[i, 'm, really, suprised, this, movie, did, n'..."
4,I'll start by confessing that I tend to really...,"[I, 'll, start, by, confessing, that, I, tend,...","[i, 'll, start, by, confessing, that, i, tend,..."
...,...,...,...
34995,`Shadow Magic' recaptures the joy and amazemen...,"[`, Shadow, Magic, ', recaptures, the, joy, an...","[`, shadow, magic, ', recaptures, the, joy, an..."
34996,I found this movie to be quite enjoyable and f...,"[I, found, this, movie, to, be, quite, enjoyab...","[i, found, this, movie, to, be, quite, enjoyab..."
34997,Avoid this one! It is a terrible movie. So wha...,"[Avoid, this, one, !, It, is, a, terrible, mov...","[avoid, this, one, !, it, is, a, terrible, mov..."
34998,This production was quite a surprise for me. I...,"[This, production, was, quite, a, surprise, fo...","[this, production, was, quite, a, surprise, fo..."


In [116]:
tokens_vocab_size = len(set(token for tokens in X_train_df['tokens'] for token in tokens))
lemmas_vocab_size = len(set(lemma for lemmas in X_train_df['lemmas'] for lemma in lemmas))
print("Taille du vocabulaire pour les tokens  :", tokens_vocab_size)
print("Taille du vocabulaire pour les lemmes :", lemmas_vocab_size)

Taille du vocabulaire pour les tokens  : 146293
Taille du vocabulaire pour les lemmes : 121716


In [138]:
X_valid_df['tokens'] = X_valid_df['review'].apply(tokenize_text)

/home/moustapha/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [139]:
X_valid_df

,review,tokens
33553,I really liked this Summerslam due to the look...,"[I, really, liked, this, Summerslam, due, to, ..."
9427,Not many television shows appeal to quite as m...,"[Not, many, television, shows, appeal, to, qui..."
199,The film quickly gets to a major chase scene w...,"[The, film, quickly, gets, to, a, major, chase..."
12447,Jane Austen would definitely approve of this o...,"[Jane, Austen, would, definitely, approve, of,..."
39489,Expectations were somewhat high for me when I ...,"[Expectations, were, somewhat, high, for, me, ..."
...,...,...
15168,"""Landscape after a battle"" opens with escaping...","["", Landscape, after, a, battle, "", opens, wit..."
49241,Jake Speed (1986) was an amusing parody of Ind...,"[Jake, Speed, (, 1986, ), was, an, amusing, pa..."
39317,"PLAN B has the appearance of a quickly made, u...","[PLAN, B, has, the, appearance, of, a, quickly..."
42191,One of the perks of my job is that when things...,"[One, of, the, perks, of, my, job, is, that, w..."


#### Bag-of-Words" (BoW) vector representation

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=2, max_features=None, stop_words=None)
X_train_tfidf = tfidf_vectorizer.fit_transform([" ".join(doc) for doc in X_train_df["tokens"]])
X_valid_tfidf = tfidf_vectorizer.transform([" ".join(doc) for doc in X_valid_df["tokens"]])


### Apprentissage via une régression logistique 

In [144]:
# instance de classifieur de type "régression logistique"

clf_f = LogisticRegression(
        #random_state=0, 
        solver='lbfgs',    # algo d'optimisation (ici minimisation de la perte cross-entropie)
        multi_class='ovr', # stratégie "one versus rest"
        penalty='l2', # hyperparamètre de régularisation
        C=1.0, # hyperparamètre de régularisation
        n_jobs=2
)

clf_f.fit(X_train_tfidf, y_train)

LogisticRegression(multi_class='ovr', n_jobs=2)

In [148]:
Y_valid_pred_f = clf_f.predict(X_valid_tfidf)
print("SCORE OF LOGISTIC REGRESSION ON VALID: %.3f" % accuracy_score(y_valid, Y_valid_pred_f))

SCORE OF LOGISTIC REGRESSION ON VALID: 0.901
